# Model Tuning: Grid Search + Pipeline

In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Objectives

- Explain what hyperparameters are
- Describe the purpose of grid searching
- Implement grid searching for the purposes of model optimization.

# Model Tuning

![](https://imgs.xkcd.com/comics/machine_learning.png)

## Hyperparameters

Many of the models we have looked at are really *families* of models in the sense that they make use of **hyperparameters**.

Thus for example the $k$-nearest-neighbors algorithm allows us to make:

- a 1-nearest-neighbor model
- a 2-nearest-neighbors model
- a 3-nearest-neighbors model
- etc.

Or, for another example, the decision tree algorithm allows us to make:

- a classifier that branches according to information gain
- a classifier that branches according to Gini impurity
- a regressor that branches according to mean squared error
- etc.

Depending on the sort of problem and data at hand, it is natural to experiment with different values of these hyperparameters to try to improve model performance.

> We can think of these **hyperparameters** as _dials_ of the base model

<img width=60% src='images/dials.png'/>

### Difference from Parametric / Non-Parametric Models

Contrast the notion of hyperparameters with the distinction between parametric and non-parametric models.

A linear regression model is parametric in the sense that we start with a given model *form* and we then search for the optimal parameters to fill in that form. But *those* parameters are not the sort we might tweak for the purposes of improving model performance. On the contrary, there is one best set of parameters, and the training of the model is a matter of finding those optimal values.

## Data Example

![Penguins](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/69530276d74b99df81cc385f4e95c644da69ebfa/man/figures/lter_penguins.png)

> Images source: @allison_horst [github.com/allisonhorst/penguins](github.com/allisonhorst/penguins)

In [11]:
penguins = sns.load_dataset('penguins')

![Bill length & depth](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/69530276d74b99df81cc385f4e95c644da69ebfa/man/figures/culmen_depth.png)

> Images source: @allison_horst [github.com/allisonhorst/penguins](github.com/allisonhorst/penguins)

In [12]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [13]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [14]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


### Data Prep

We'll try to predict species given the other columns' values. Let's dummy-out `island` and `sex`:

In [17]:
penguins.isna().sum().sum()

0

In [16]:
penguins = penguins.dropna()

In [18]:
y = penguins.pop('species')
y

0      Adelie
1      Adelie
2      Adelie
4      Adelie
5      Adelie
        ...  
338    Gentoo
340    Gentoo
341    Gentoo
342    Gentoo
343    Gentoo
Name: species, Length: 333, dtype: object

In [19]:
penguins

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...
338,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Biscoe,45.2,14.8,212.0,5200.0,Female


In [20]:
# Note we're dedicating a lot of data to the testing set just for demonstrative purposes
X_train, X_test, y_train, y_test = train_test_split(
    penguins, y, test_size=0.5, random_state=42)

In [21]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [22]:
dums_df.head()

,x0_Dream,x0_Torgersen,x1_Male
160,1.0,0.0,0.0
237,0.0,0.0,1.0
2,0.0,1.0,0.0
121,0.0,1.0,1.0
179,1.0,0.0,1.0


In [24]:
X_train_nums = X_train.select_dtypes('float64')

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)
nums_df

,0,1,2,3
160,0.362748,0.903276,-0.472344,-0.094599
237,0.973499,-0.977375,1.408317,2.512546
2,-0.725152,0.445820,-0.472344,-1.185963
121,-1.221387,1.360731,-0.255345,-0.882806
179,1.030757,0.954104,-0.110678,-0.519018
...,...,...,...,...
194,1.297960,1.004932,-0.400011,-0.822175
77,-1.316816,1.157417,-1.268008,-0.397756
112,-0.839667,0.293335,-0.617010,-1.246594
277,0.267318,-1.079032,1.335984,0.936132


In [25]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

In [26]:
X_train_clean.head()

,0,1,2,3,x0_Dream,x0_Torgersen,x1_Male
160,0.362748,0.903276,-0.472344,-0.094599,1.0,0.0,0.0
237,0.973499,-0.977375,1.408317,2.512546,0.0,0.0,1.0
2,-0.725152,0.445820,-0.472344,-1.185963,0.0,1.0,0.0
121,-1.221387,1.360731,-0.255345,-0.882806,0.0,1.0,1.0
179,1.030757,0.954104,-0.110678,-0.519018,1.0,0.0,1.0


#### Preparing the Test Set

In [27]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names(),
                      index=X_test_cat.index)

In [28]:
X_test_nums = X_test.select_dtypes('float64')

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                           index=X_test_nums.index)

In [29]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df], axis=1)

In [30]:
X_test_clean.head()

,0,1,2,3,x0_Dream,x0_Torgersen,x1_Male
30,-0.877839,-0.214949,-1.702007,-1.185963,1.0,0.0,0.0
317,0.534522,-1.282345,1.480650,0.784554,0.0,0.0,0.0
79,-0.381604,1.004932,-0.472344,-0.276493,0.0,1.0,1.0
201,1.088015,0.090021,-0.255345,-0.670597,1.0,0.0,0.0
63,-0.572464,0.547477,-0.689343,-0.215862,0.0,0.0,1.0


### Trying Different Models & Values

#### $k$-Nearest Neighbors Model

In [31]:
knn_model = KNeighborsClassifier()

knn_model.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [32]:
scores = cross_val_score(estimator=knn_model, X=X_train_clean,
               y=y_train, cv=10)
scores

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       0.94117647, 1.        , 1.        , 1.        , 1.        ])

In [33]:
np.median(scores)

1.0

In [34]:
knn_model.score(X_test_clean, y_test)

0.9880239520958084

##### Decreasing $k$

In [43]:
knn3 = KNeighborsClassifier(n_neighbors=3)

knn3.fit(X_train_clean, y_train)

KNeighborsClassifier(n_neighbors=3)

In [44]:
knn3.score(X_test_clean, y_test)

0.9940119760479041

#### Decision Tree

In [45]:
ct = DecisionTreeClassifier(random_state=10)

ct.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=10)

In [46]:
ct.score(X_test_clean, y_test)

0.9760479041916168

##### Changing the branching criterion

In [47]:
ct = DecisionTreeClassifier(criterion='entropy',
                          random_state=10)

ct.fit(X_train_clean, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [48]:
ct.score(X_test_clean, y_test)

0.9760479041916168

# Automatically Searching with Grid Search

It's not a bad idea to experiment with the values of your models' hyperparameters a bit as you're getting a feel for your models' performance. But there are more systematic ways of going about the search for optimal hyperparameters. One method of hyperparameter tuning is **grid searching**. 

The idea is to build multiple models with different hyperparameter values and then see which one performs the best. The hyperparameters and the values to try form a sort of *grid* along which we are looking for the best performance. For example:


    1           | 'minkowski' | 'uniform'
    3           | 'manhattan' | 'distance'
    5           |
    ______________________________________
    n_neighbors | metric      | weights

Scikit-Learn has a [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class whose `fit()` method runs this procedure. Note that this can be quite computationally expensive since:

- A model is constructed for each combination of hyperparameter values that we input; and
- Each model is cross-validated.

In [50]:
3 * 2 * 2 * 5

60

### `GridSearchCV`

In [51]:
# Define the parameter grid

grid = {'n_neighbors': [1, 3, 5],
       'metric': ['minkowski', 'manhattan'],
       'weights': ['uniform', 'distance']}

**Question: How many models will we be constructing with this grid?**

In [75]:
# Initialize the grid search object with five-fold cross-validation

gs = GridSearchCV(estimator=knn_model, param_grid=grid, verbose=2)

In [76]:
gs.fit(X_train_clean, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] metric=minkowski, n_neighbors=1, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=minkowski, n_neighbors=1, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=minkowski, n_neighbors=1, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=minkowski, n_neighbors=1, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=minkowski, n_neighbors=1, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=minkowski, n_neighbors=1, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=1, weights=distance, total=   0.0s
[CV] metric=minkowski, n_neighbors=1, weights=distance ...............
[CV]  metric=min

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  metric=manhattan, n_neighbors=3, weights=distance, total=   0.0s
[CV] metric=manhattan, n_neighbors=3, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=3, weights=distance, total=   0.0s
[CV] metric=manhattan, n_neighbors=3, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=3, weights=distance, total=   0.0s
[CV] metric=manhattan, n_neighbors=3, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=3, weights=distance, total=   0.0s
[CV] metric=manhattan, n_neighbors=5, weights=uniform ................
[CV] . metric=manhattan, n_neighbors=5, weights=uniform, total=   0.0s
[CV] metric=manhattan, n_neighbors=5, weights=uniform ................
[CV] . metric=manhattan, n_neighbors=5, weights=uniform, total=   0.0s
[CV] metric=manhattan, n_neighbors=5, weights=uniform ................
[CV] . metric=manhattan, n_neighbors=5, weights=uniform, total=   0.0s
[CV] metric=manhattan, n_neighbors=5, weights=uniform ................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.2s finished


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski', 'manhattan'],
                         'n_neighbors': [1, 3, 5],
                         'weights': ['uniform', 'distance']},
             verbose=2)

In [66]:
gs.best_params_

{'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}

In [67]:
gs.best_score_

0.9939393939393939

In [69]:
gs.best_estimator_.score(X_test_clean, y_test)

0.9940119760479041

In [70]:
final_model = gs.best_index_

In [71]:
gs.cv_results_

{'mean_fit_time': array([0.00337934, 0.00198727, 0.00152802, 0.00124826, 0.00111966,
        0.00124803, 0.00111523, 0.00117135, 0.00118923, 0.00114727,
        0.00113935, 0.00127907]),
 'std_fit_time': array([1.34162665e-03, 2.63550205e-04, 1.48674584e-04, 8.18745647e-05,
        6.81045332e-05, 2.18742860e-04, 9.29686462e-05, 8.12742324e-05,
        1.89670454e-04, 7.16400819e-05, 2.42463138e-05, 2.58516306e-04]),
 'mean_score_time': array([0.00521016, 0.00197277, 0.0026041 , 0.00133886, 0.00182695,
        0.00136561, 0.00176663, 0.00127077, 0.00187721, 0.00127897,
        0.00189958, 0.00161119]),
 'std_score_time': array([1.29782156e-03, 2.32171253e-04, 2.91774183e-04, 6.90849008e-05,
        1.33961661e-04, 4.68436146e-05, 1.03669750e-04, 1.11838900e-04,
        6.71536052e-05, 9.93074029e-05, 2.21017166e-04, 3.20256182e-04]),
 'param_metric': masked_array(data=['minkowski', 'minkowski', 'minkowski', 'minkowski',
                    'minkowski', 'minkowski', 'manhattan', 'manhat

In [72]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003379,0.001342,0.005210,0.001298,minkowski,1,uniform,"{'metric': 'minkowski', 'n_neighbors': 1, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
1,0.001987,0.000264,0.001973,0.000232,minkowski,1,distance,"{'metric': 'minkowski', 'n_neighbors': 1, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
2,0.001528,0.000149,0.002604,0.000292,minkowski,3,uniform,"{'metric': 'minkowski', 'n_neighbors': 3, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
3,0.001248,0.000082,0.001339,0.000069,minkowski,3,distance,"{'metric': 'minkowski', 'n_neighbors': 3, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
4,0.001120,0.000068,0.001827,0.000134,minkowski,5,uniform,"{'metric': 'minkowski', 'n_neighbors': 5, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
5,0.001248,0.000219,0.001366,0.000047,minkowski,5,distance,"{'metric': 'minkowski', 'n_neighbors': 5, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
6,0.001115,0.000093,0.001767,0.000104,manhattan,1,uniform,"{'metric': 'manhattan', 'n_neighbors': 1, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
7,0.001171,0.000081,0.001271,0.000112,manhattan,1,distance,"{'metric': 'manhattan', 'n_neighbors': 1, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
8,0.001189,0.000190,0.001877,0.000067,manhattan,3,uniform,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1
9,0.001147,0.000072,0.001279,0.000099,manhattan,3,distance,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei...",1.0,1.0,0.969697,1.0,1.0,0.993939,0.012121,1


### Choice of Grid Values

Which values should you pick for your grid? Intuitively, you should try both "large" and "small" values, but of course what counts as large and small will really depend on the type of hyperparameter.

- ALWAYS INCLUDE THE DEFAULT IN YOUR FIRST SEARCH
- For a k-nearest neighbors model, 1 or 3 would be a small value for the number of neighbors and 15 or 17 would be a large value.
- For a decision tree model, what counts as a small `max_depth` will really depend on the size of your training data. A `max_depth` of 5 would likely have little effect on a very small dataset but, at the same time, it would probably significantly decrease the variance of a model where the dataset is large.
- For a logistic regression's regularization constant, you may want to try a set of values that are exponentially separated, like \[1, 10, 100, 1000\].
- **If a grid search finds optimal values at the ends of your hyperparameter ranges, you might try another grid search with more extreme values.**

In [77]:
#n_neighbors_grid = [1, 3, 5, 7, 9] - best param was 9
#next grid search might run somehting like [9, 11, 13, 15, 17]

### Exercise

Do a grid search on a **decision tree model** of penguin species. What are the optimal values for the hyperparameters you've chosen?

# Better Process: Pipelines

> **Pipelines** can keep our code neat and clean all the way from gathering & cleaning our data, to creating models & fine-tuning them!

![](https://imgs.xkcd.com/comics/data_pipeline.png)

The `Pipeline` class from [Scikit-Learn's API](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) is especially convenient since it allows us to use our other Estimators that we know and love!

## Advantages of `Pipeline`

### Reduces Complexity

> You can focus on particular parts of the pipeline one at a time and debug or adjust parts as needed.

### Convenient

> The pipeline summarizes your fine-detail steps. That way you can focus on the big-picture aspects.

### Flexible

> You can use pipelines with different models and with GridSearch.

### Prevent Mistakes

> We can focus on one section at a time.
>
> We also can ensure data leakage between our training and doesn't occur between our training dataset and validation/testing datasets!

## Example of Using `Pipeline`

In [78]:
# Getting some data
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=27)

### Without the Pipeline class

In [79]:
# Define transformers (will adjust/massage the data)
imputer = SimpleImputer(strategy="median") # replaces missing values
std_scaler = StandardScaler() # scales the data

# Define the classifier (predictor) to train
rf_clf = DecisionTreeClassifier(random_state=42)

# Have the classifer (and full pipeline) learn/train/fit from the data
X_train_filled = imputer.fit_transform(X_train)
X_train_scaled = std_scaler.fit_transform(X_train_filled)
rf_clf.fit(X_train_scaled, y_train)

# Predict using the trained classifier (still need to do the transformations)
X_test_filled = imputer.transform(X_test)
X_test_scaled = std_scaler.transform(X_test_filled)
y_pred = rf_clf.predict(X_test_scaled)
print(y_pred)

[2 0 2 2 1 1 1 2 2 0 1 1 0 1 1 1 0 2 2 2 1 0 1 0 2 2 1 0 1 2]


> Note that if we were to add more steps in this process, we'd have to change both the *training* and *testing* processes.

### With `Pipeline` Class

In [80]:
pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), 
        ('std_scaler', StandardScaler()),
        ('rf_clf', DecisionTreeClassifier(random_state=42)),
])


# Train the pipeline (tranformations & predictor)
pipeline.fit(X_train, y_train)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted = pipeline.predict(X_test)
print(predicted)

[2 0 2 2 1 1 1 2 2 0 1 1 0 1 1 1 0 2 2 2 1 0 1 0 2 2 1 0 1 2]


In [81]:
pipeline['imputer']

SimpleImputer(strategy='median')

In [83]:
pipeline.named_steps

{'imputer': SimpleImputer(strategy='median'),
 'std_scaler': StandardScaler(),
 'rf_clf': DecisionTreeClassifier(random_state=42)}

In [85]:
pipeline['rf_clf'].feature_importances_

array([0.        , 0.03338202, 0.55667565, 0.40994234])

> If we need to change our process, we change it _just once_ in the Pipeline

## Grid Searching a Pipeline

> Let's first get our data prepared like we did before

In [86]:
penguins = sns.load_dataset('penguins')
penguins = penguins.dropna()

In [87]:
y = penguins.pop('species')
X_train, X_test, y_train, y_test = train_test_split(
    penguins, y, test_size=0.5, random_state=42)

In [88]:
X_train_nums = X_train.select_dtypes('float64')

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [89]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

> Intermediary step to treat categorical and numerical data differently

### Using `ColumnTransformer`

In [90]:
X_train_nums.columns

Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')

In [98]:
numerical_pipeline = Pipeline(steps=[('ss', StandardScaler())])

categorical_pipeline = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, 
                                                            handle_unknown='ignore'))])
transformer = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, X_train_nums.columns),
    ('cat', categorical_pipeline, X_train_cat.columns)])

In [120]:
transformer2 = ColumnTransformer(transformers=[
    ('num', StandardScaler(), X_train_nums.columns),
    ('cat', categorical_pipeline, X_train_cat.columns)])

In [99]:
model_pipe = Pipeline(steps=[('col_tr', transformer),
                             ('knn', KNeighborsClassifier())])

In [121]:
model_pipe2 = Pipeline(steps=[('col_tr', transformer2),
                             ('knn', KNeighborsClassifier())])

> Finally showing we can fit the full pipeline

In [122]:
model_pipe.fit(X_train, y_train)

Pipeline(steps=[('col_tr',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['island', 'sex'], dtype='object'))])),
                ('knn', KNeighborsClassifier())])

In [123]:
model_pipe.score(X_test, y_test)

0.9880239520958084

In [100]:
model_pipe.fit(X_train, y_train)

Pipeline(steps=[('col_tr',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['island', 'sex'], dtype='object'))])),
                ('knn', KNeighborsClassifier())])

In [101]:
model_pipe.score(X_train, y_train)

0.9939759036144579

In [102]:
model_pipe.score(X_test, y_test)

0.9880239520958084

> Performing grid search on the full pipeline

In [117]:
model_pipe.named_steps

{'col_tr': ColumnTransformer(transformers=[('num',
                                  Pipeline(steps=[('ss', StandardScaler())]),
                                  Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')),
                                 ('cat',
                                  Pipeline(steps=[('ohe',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse=False))]),
                                  Index(['island', 'sex'], dtype='object'))]),
 'knn': KNeighborsClassifier()}

In [105]:
model_pipe.named_steps['knn']

KNeighborsClassifier()

In [119]:
model_pipe['col_tr'].named_transformers_

{'num': Pipeline(steps=[('ss', StandardScaler())]),
 'cat': Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))])}

In [130]:
pipe_grid = {'knn__n_neighbors': [3, 5, 7],
             'knn__p': [1, 2, 3],
             'col_tr__num__ss__with_std': [True, False]}
gs_pipe = GridSearchCV(estimator=model_pipe, param_grid=pipe_grid, verbose=2)

In [131]:
gs_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=2 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=3, knn__p=2, total=   0.0s
[CV] col_tr__num__ss_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=1 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=1, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2, total=   0.0s
[CV] col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2 ....
[CV]  col_tr__num__ss__with_std=True, knn__n_neighbors=5, knn__p=2, total=   0.0s


[CV]  col_tr__num__ss__with_std=False, knn__n_neighbors=7, knn__p=3, total=   0.0s
[CV] col_tr__num__ss__with_std=False, knn__n_neighbors=7, knn__p=3 ...
[CV]  col_tr__num__ss__with_std=False, knn__n_neighbors=7, knn__p=3, total=   0.0s


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    0.9s finished


GridSearchCV(estimator=Pipeline(steps=[('col_tr',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('ss',
                                                                                          StandardScaler())]),
                                                                         Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                 

In [126]:
pd.DataFrame(gs_pipe.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_col_tr__num__ss__with_std,param_knn__n_neighbors,param_knn__p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009543,0.003730,0.005535,0.001335,True,3,1,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,1.000000,1.000000,0.993939,0.012121,1
1,0.004192,0.000275,0.003305,0.000227,True,3,2,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,1.000000,1.000000,0.993939,0.012121,1
2,0.004638,0.000625,0.003954,0.000443,True,3,3,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,1.000000,1.000000,0.993939,0.012121,1
3,0.004978,0.001066,0.003545,0.000213,True,5,1,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,1.000000,1.000000,0.993939,0.012121,1
4,0.005766,0.001638,0.004371,0.000835,True,5,2,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,1.000000,1.000000,0.993939,0.012121,1
5,0.004214,0.000135,0.003690,0.000950,True,5,3,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,1.000000,1.000000,0.993939,0.012121,1
6,0.004389,0.000435,0.003430,0.000314,True,7,1,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,0.969697,1.000000,0.987879,0.014845,7
7,0.004228,0.000631,0.003345,0.000132,True,7,2,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,0.969697,1.000000,0.987879,0.014845,7
8,0.005027,0.000458,0.004827,0.001534,True,7,3,"{'col_tr__num__ss__with_std': True, 'knn__n_ne...",1.000000,1.000000,0.969697,0.969697,1.000000,0.987879,0.014845,7
9,0.004229,0.000354,0.003850,0.000687,False,3,1,"{'col_tr__num__ss__with_std': False, 'knn__n_n...",0.794118,0.727273,0.727273,0.515152,0.878788,0.728520,0.120343,10


In [127]:
gs_pipe.best_params_

{'col_tr__num__ss__with_std': True, 'knn__n_neighbors': 3, 'knn__p': 1}

## A Note on Data Leakage

In [133]:
gs_pipe.best_estimator_.score(X_test, y_test)

0.9940119760479041

Note we still have to be careful in performing a grid search!

We can accidentally "leak" information by doing transformations with the **whole data set**, instead of just the **training set**!

### Example of leaking information

In [134]:
scaler = StandardScaler()
# Scales over all of the X-train data! (validation set will be considered in scaling)
scaled_data = scaler.fit_transform(X_train.select_dtypes('float64'))

parameters = {
    'n_neighbors': [1, 3, 5],
    'metric': ['minkowski', 'manhattan'],
    'weights': ['uniform', 'distance']
}

clf_dt = KNeighborsClassifier()
clf = GridSearchCV(clf_dt, parameters)
clf.fit(X_train.select_dtypes('float64'), y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski', 'manhattan'],
                         'n_neighbors': [1, 3, 5],
                         'weights': ['uniform', 'distance']})

### Example of Grid Search with no leakage

In [135]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

# Note you use the part of the pipeline's name `NAME__{parameter}`
parameters = {
    'scaler__with_mean': [True, False],
    'clf__n_neighbors': [1, 3, 5],
    'clf__metric': ['minkowski', 'manhattan'],
    'clf__weights': ['uniform', 'distance']
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train.select_dtypes('float64'), y_train)
y_pred = cv.predict(X_test.select_dtypes('float64'))
print(y_pred)

['Adelie' 'Gentoo' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo' 'Gentoo'
 'Chinstrap' 'Chinstrap' 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Adelie'
 'Gentoo' 'Adelie' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo' 'Adelie'
 'Adelie' 'Gentoo' 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Gentoo'
 'Chinstrap' 'Gentoo' 'Chinstrap' 'Gentoo' 'Adelie' 'Adelie' 'Gentoo'
 'Gentoo' 'Chinstrap' 'Gentoo' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Chinstrap' 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Adelie' 'Adelie'
 'Gentoo' 'Adelie' 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Gentoo'
 'Chinstrap' 'Adelie' 'Chinstrap' 'Chinstrap' 'Gentoo' 'Gentoo' 'Gentoo'
 'Adelie' 'Adelie' 'Gentoo' 'Adelie' 'Gentoo' 'Adelie' 'Gentoo' 'Adelie'
 'Adelie' 'Gentoo' 'Gentoo' 'Gentoo' 'Chinstrap' 'Adelie' 'Adelie'
 'Adelie' 'Gentoo' 'Chinstrap' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo'
 'Gentoo' 'Adelie' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Gentoo' 'Gentoo'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Chinstrap' 'Adelie'
 'Adelie' 'Adelie' 'Gentoo' 'Gentoo

# Grid Search Exercise

Use a classifier of your choice to predict the category of price range for the phones in this dataset. Try tuning some hyperparameters using a grid search, and then write up a short paragraph about your findings.

In [ ]:
phones_train = pd.read_csv('data/train.csv')
phones_test = pd.read_csv('data/test.csv')

# Level Up: Random Searching

It is also possible to search for good hyperparameter values randomly. This is a nice choice if computation time is an issue or if you are tuning over continuous hyperparameters.

### `RandomizedSearchCV` with `LogisticRegression`

In [137]:
log_reg_grid = {'C': stats.uniform(loc=0, scale=10),
               'l1_ratio': stats.expon(scale=0.2)}

In [143]:
rs = RandomizedSearchCV(estimator=LogisticRegression(penalty='elasticnet',
                                                    solver='saga',
                                                    max_iter=1000,
                                                    random_state=42),
                        param_distributions=log_reg_grid,
                       random_state=42, n_iter=100, verbose=2)

rs.fit(X_train_clean, y_train)

rs.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] C=3.745401188473625, l1_ratio=0.6020242861835042 ................
[CV] . C=3.745401188473625, l1_ratio=0.6020242861835042, total=   0.0s
[CV] C=3.745401188473625, l1_ratio=0.6020242861835042 ................
[CV] . C=3.745401188473625, l1_ratio=0.6020242861835042, total=   0.0s
[CV] C=3.745401188473625, l1_ratio=0.6020242861835042 ................
[CV] . C=3.745401188473625, l1_ratio=0.6020242861835042, total=   0.0s
[CV] C=3.745401188473625, l1_ratio=0.6020242861835042 ................
[CV] . C=3.745401188473625, l1_ratio=0.6020242861835042, total=   0.0s
[CV] C=3.745401188473625, l1_ratio=0.6020242861835042 ................
[CV] . C=3.745401188473625, l1_ratio=0.6020242861835042, total=   0.0s
[CV] C=7.319939418114051, l1_ratio=0.18258851075519067 ...............
[CV]  C=7.319939418114051, l1_ratio=0.18258851075519067, total=   0.0s
[CV] C=7.319939418114051, l1_ratio=0.18258851075519067 ...............
[CV]  C=7.3199

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=6.011150117432088, l1_ratio=0.24625001234091806, total=   0.0s
[CV] C=6.011150117432088, l1_ratio=0.24625001234091806 ...............
[CV]  C=6.011150117432088, l1_ratio=0.24625001234091806, total=   0.0s
[CV] C=0.20584494295802447, l1_ratio=0.7007114950316624 ..............
[CV]  C=0.20584494295802447, l1_ratio=0.7007114950316624, total=   0.0s
[CV] C=0.20584494295802447, l1_ratio=0.7007114950316624 ..............
[CV]  C=0.20584494295802447, l1_ratio=0.7007114950316624, total=   0.0s
[CV] C=0.20584494295802447, l1_ratio=0.7007114950316624 ..............
[CV]  C=0.20584494295802447, l1_ratio=0.7007114950316624, total=   0.0s
[CV] C=0.20584494295802447, l1_ratio=0.7007114950316624 ..............
[CV]  C=0.20584494295802447, l1_ratio=0.7007114950316624, total=   0.0s
[CV] C=0.20584494295802447, l1_ratio=0.7007114950316624 ..............
[CV]  C=0.20584494295802447, l1_ratio=0.7007114950316624, total=   0.0s
[CV] C=8.324426408004218, l1_ratio=0.0477375250497894 ................
[

[CV]  C=9.656320330745594, l1_ratio=0.33046631456257536, total=   0.0s
[CV] C=9.656320330745594, l1_ratio=0.33046631456257536 ...............
[CV]  C=9.656320330745594, l1_ratio=0.33046631456257536, total=   0.0s
[CV] C=9.656320330745594, l1_ratio=0.33046631456257536 ...............
[CV]  C=9.656320330745594, l1_ratio=0.33046631456257536, total=   0.0s
[CV] C=9.656320330745594, l1_ratio=0.33046631456257536 ...............
[CV]  C=9.656320330745594, l1_ratio=0.33046631456257536, total=   0.0s
[CV] C=9.656320330745594, l1_ratio=0.33046631456257536 ...............
[CV]  C=9.656320330745594, l1_ratio=0.33046631456257536, total=   0.0s
[CV] C=3.0461376917337066, l1_ratio=0.020555463000501248 .............
[CV]  C=3.0461376917337066, l1_ratio=0.020555463000501248, total=   0.0s
[CV] C=3.0461376917337066, l1_ratio=0.020555463000501248 .............
[CV]  C=3.0461376917337066, l1_ratio=0.020555463000501248, total=   0.0s
[CV] C=3.0461376917337066, l1_ratio=0.020555463000501248 .............
[C

[CV]  C=7.722447692966574, l1_ratio=0.044307888101177195, total=   0.0s
[CV] C=7.722447692966574, l1_ratio=0.044307888101177195 ..............
[CV]  C=7.722447692966574, l1_ratio=0.044307888101177195, total=   0.0s
[CV] C=7.722447692966574, l1_ratio=0.044307888101177195 ..............
[CV]  C=7.722447692966574, l1_ratio=0.044307888101177195, total=   0.0s
[CV] C=0.05522117123602399, l1_ratio=0.33797935549722524 .............
[CV]  C=0.05522117123602399, l1_ratio=0.33797935549722524, total=   0.0s
[CV] C=0.05522117123602399, l1_ratio=0.33797935549722524 .............
[CV]  C=0.05522117123602399, l1_ratio=0.33797935549722524, total=   0.0s
[CV] C=0.05522117123602399, l1_ratio=0.33797935549722524 .............
[CV]  C=0.05522117123602399, l1_ratio=0.33797935549722524, total=   0.0s
[CV] C=0.05522117123602399, l1_ratio=0.33797935549722524 .............
[CV]  C=0.05522117123602399, l1_ratio=0.33797935549722524, total=   0.0s
[CV] C=0.05522117123602399, l1_ratio=0.33797935549722524 .........

...

[CV] .. C=4.17411003148779, l1_ratio=0.0502334676551649, total=   0.0s
[CV] C=1.198653673336828, l1_ratio=0.0823817159724865 ................
[CV] . C=1.198653673336828, l1_ratio=0.0823817159724865, total=   0.0s
[CV] C=1.198653673336828, l1_ratio=0.0823817159724865 ................
[CV] . C=1.198653673336828, l1_ratio=0.0823817159724865, total=   0.0s
[CV] C=1.198653673336828, l1_ratio=0.0823817159724865 ................
[CV] . C=1.198653673336828, l1_ratio=0.0823817159724865, total=   0.0s
[CV] C=1.198653673336828, l1_ratio=0.0823817159724865 ................
[CV] . C=1.198653673336828, l1_ratio=0.0823817159724865, total=   0.0s
[CV] C=1.198653673336828, l1_ratio=0.0823817159724865 ................
[CV] . C=1.198653673336828, l1_ratio=0.0823817159724865, total=   0.0s
[CV] C=9.429097039125192, l1_ratio=0.07807676057629488 ...............
[CV]  C=9.429097039125192, l1_ratio=0.07807676057629488, total=   0.0s
[CV] C=9.429097039125192, l1_ratio=0.07807676057629488 ...............
[CV]  

[CV]  C=3.677831327192532, l1_ratio=0.20010074954635426, total=   0.0s
[CV] C=3.677831327192532, l1_ratio=0.20010074954635426 ...............
[CV]  C=3.677831327192532, l1_ratio=0.20010074954635426, total=   0.0s
[CV] C=6.335297107608947, l1_ratio=0.15347704999046977 ...............
[CV]  C=6.335297107608947, l1_ratio=0.15347704999046977, total=   0.0s
[CV] C=6.335297107608947, l1_ratio=0.15347704999046977 ...............
[CV]  C=6.335297107608947, l1_ratio=0.15347704999046977, total=   0.0s
[CV] C=6.335297107608947, l1_ratio=0.15347704999046977 ...............
[CV]  C=6.335297107608947, l1_ratio=0.15347704999046977, total=   0.0s
[CV] C=6.335297107608947, l1_ratio=0.15347704999046977 ...............
[CV]  C=6.335297107608947, l1_ratio=0.15347704999046977, total=   0.0s
[CV] C=6.335297107608947, l1_ratio=0.15347704999046977 ...............
[CV]  C=6.335297107608947, l1_ratio=0.15347704999046977, total=   0.0s
[CV] C=0.902897700544083, l1_ratio=0.36072895884925943 ...............
[CV]  

[CV]  C=9.004180571633304, l1_ratio=0.20053398389657695, total=   0.0s
[CV] C=9.004180571633304, l1_ratio=0.20053398389657695 ...............
[CV]  C=9.004180571633304, l1_ratio=0.20053398389657695, total=   0.0s
[CV] C=9.004180571633304, l1_ratio=0.20053398389657695 ...............
[CV]  C=9.004180571633304, l1_ratio=0.20053398389657695, total=   0.0s
[CV] C=9.004180571633304, l1_ratio=0.20053398389657695 ...............
[CV]  C=9.004180571633304, l1_ratio=0.20053398389657695, total=   0.0s
[CV] C=9.004180571633304, l1_ratio=0.20053398389657695 ...............
[CV]  C=9.004180571633304, l1_ratio=0.20053398389657695, total=   0.0s
[CV] C=3.390297910487007, l1_ratio=0.0859135231623288 ................
[CV] . C=3.390297910487007, l1_ratio=0.0859135231623288, total=   0.0s
[CV] C=3.390297910487007, l1_ratio=0.0859135231623288 ................
[CV] . C=3.390297910487007, l1_ratio=0.0859135231623288, total=   0.0s
[CV] C=3.390297910487007, l1_ratio=0.0859135231623288 ................
[CV] .

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    3.9s finished


{'C': 3.745401188473625, 'l1_ratio': 0.6020242861835042}

In [140]:
stats.expon(scale=0.2).rvs(10)

array([0.08427088, 0.44513906, 0.27246846, 0.08580529, 0.19496272,
       0.78948944, 0.30045399, 0.82703738, 0.30384035, 0.53730601])

In [141]:
stats.uniform(loc=0, scale=10).rvs(10)

array([6.87455897, 4.02010467, 0.38304807, 1.2581645 , 1.190306  ,
       8.95795658, 2.90043981, 7.12422138, 1.6799912 , 5.0641189 ])

# Level Up: SMOTE

Often we encounter a problem of imbalance classification that there are too few observations of the minority class for a model to effectively learn the decision boundary. 

One way to solve this problem is to **oversample** the observations in the minority class (or alternatively **undersample** the observations in the majority class) by synthesizing new observation from the minority class.

The most widely used approach to synthesizing new observations is called the **Synthetic Minority Oversampling Technique**, or **SMOTE** for short. 

Resource: https://arxiv.org/abs/1106.1813

Before getting into the example, please note the following,

1. Oversampling process is based on **k-nearest neighbors** of the minority class.
2. Oversampling only works with **numerical predictors** since the synthetic observations are created based on the k-nearest neighbors algorithm, which is a distance based algorithm.  

We use the Scikit-Learn's breast cancer dataset to demonstrate the use of SMOTE from imblearn package.

In [144]:
from sklearn.datasets import load_breast_cancer

# Load the data
preds, target = load_breast_cancer(return_X_y=True)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(preds, target,
                                                   random_state=42)

In [145]:
# Import imblearn dependencies
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline
from collections import Counter
from matplotlib import pyplot
from numpy import where

Note that we have an imbalance class for the target variable in this dataset.

In [146]:
# Check the class distribution of the target
counter = Counter(y_train)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
    row_ix = where(y_train == label)[0]
    pyplot.scatter(X_train_sc[row_ix, 0], X_train_sc[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

Counter({1: 268, 0: 158})


NameError: name 'X_train_sc' is not defined

In [149]:
# Create the oversampler and undersampler objects
over = SMOTE(sampling_strategy=0.7)
under = RandomUnderSampler(sampling_strategy=0.8)

# transform the dataset
#X, y = under.fit_resample(X_train_sc, y_train)

After the oversampling and undersampling process, we observe a more balance class distribution in the target variable.

In [ ]:
# Check the class distribution of the target
counter = Counter(y)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
    row_ix = where(y == label)[0]
    pyplot.scatter(X_train_sc[row_ix, 0], X_train_sc[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

In [150]:
# Create imblearn pipeline for the oversampler and undersampler
steps = [('o', over), ('u', under),('scale', StandardScaler()),
         ('model', DecisionTreeClassifier())]
pipeline = imbPipeline(steps=steps)


# Create the GridSearchCV object with different hyperparameters
parameters = {
    'model__max_depth': [2, 4, 6, 8],
    'model__min_samples_split': [5, 10, 15],
    'model__criterion': ['gini', 'entropy']
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

# Predict the label with the best model
y_pred = cv.predict(X_test)
print(y_pred)

[1 0 0 1 1 0 0 1 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0
 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 1 0 1]
